In [3]:

import requests
from bs4 import BeautifulSoup
import json
import time
from urllib.parse import urljoin
from datetime import datetime
import os

# Headers để tránh bị chặn
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive"
}

categories = {
    "thoi-su": "https://vnexpress.net/thoi-su-p",
    "the-gioi": "https://vnexpress.net/the-gioi-p", 
    "kinh-doanh": "https://vnexpress.net/kinh-doanh-p",
    "giai-tri": "https://vnexpress.net/giai-tri-p",
    "the-thao": "https://vnexpress.net/the-thao-p",
    "phap-luat": "https://vnexpress.net/phap-luat-p",
    "giao-duc": "https://vnexpress.net/giao-duc-p",
    "suc-khoe": "https://vnexpress.net/suc-khoe-p",
    "doi-song": "https://vnexpress.net/doi-song-p",
    "du-lich": "https://vnexpress.net/du-lich-p",
    "khoa-hoc": "https://vnexpress.net/khoa-hoc-p",
    "so-hoa": "https://vnexpress.net/so-hoa-p",
    "oto-xe-may": "https://vnexpress.net/oto-xe-may-p",
    "y-kien": "https://vnexpress.net/y-kien-p",
    "tam-su": "https://vnexpress.net/tam-su-p",
    "cuoi": "https://vnexpress.net/cuoi-p"
}

NUM_PAGES = 15  # Số trang crawl cho mỗi danh mục
all_results = []  # Lưu tất cả kết quả

def extract_article(url, category):
    try:
        r = requests.get(url, headers=headers, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        # 1. Title
        title_elem = soup.find("h1", class_="title-detail") or soup.title
        title = title_elem.get_text().strip() if title_elem else ""

        # 2. Summary/Description
        meta_desc = soup.find("meta", {"name": "description"})
        summary = meta_desc["content"].strip() if meta_desc else ""
        
        # Thử lấy sapo từ class description
        if not summary:
            sapo_elem = soup.find("p", class_="description")
            summary = sapo_elem.get_text().strip() if sapo_elem else ""

        # 3. Content - Sử dụng selector chính xác
        content = ""
        content_selectors = [
            "article.fck_detail",
            "div.fck_detail", 
            "div.sidebar-1",
            "article.content_detail"
        ]
        
        for selector in content_selectors:
            content_div = soup.select_one(selector)
            if content_div:
                paragraphs = content_div.find_all(["p", "h3", "h4", "li"])
                content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
                break

        # 4. Publish date
        pub_meta = soup.find("meta", {"itemprop": "datePublished"})
        if not pub_meta:
            pub_meta = soup.find("meta", {"property": "article:published_time"})
        publish_date = pub_meta["content"] if pub_meta else ""

        # 5. Tags - Thử nhiều cách lấy
        tags = []
        
        # Cách 1: Meta article:tag
        tag_metas = soup.find_all("meta", {"property": "article:tag"})
        for meta in tag_metas:
            if "content" in meta.attrs:
                tags.append(meta["content"].strip())
        
        # Cách 2: Meta keywords  
        keywords_meta = soup.find("meta", {"name": "keywords"})
        if keywords_meta and "content" in keywords_meta.attrs:
            keywords = [k.strip() for k in keywords_meta["content"].split(",")]
            tags.extend(keywords)
        
        # Cách 3: Meta news_keywords
        news_keywords_meta = soup.find("meta", {"name": "news_keywords"})
        if news_keywords_meta and "content" in news_keywords_meta.attrs:
            news_keywords = [k.strip() for k in news_keywords_meta["content"].split(",")]
            tags.extend(news_keywords)
            
        # Cách 4: Tags từ breadcrumb hoặc categories
        breadcrumb_links = soup.select("nav.breadcrumb a, .breadcrumb a, ul.breadcrumb a")
        for link in breadcrumb_links:
            if link.get_text().strip() and link.get_text().strip() not in ["Trang chủ", "Home"]:
                tags.append(link.get_text().strip())
        
        # Cách 5: Tags từ các thẻ div có class liên quan đến tag
        tag_divs = soup.select("div.tags a, div.tag-list a, .article-tags a")
        for tag_div in tag_divs:
            if tag_div.get_text().strip():
                tags.append(tag_div.get_text().strip())
        
        # Loại bỏ trùng lặp và làm sạch
        tags = list(set([tag for tag in tags if tag and len(tag.strip()) > 0]))
        tags = [tag for tag in tags if not tag.lower() in ["vnexpress", "vietnam", "việt nam"]]

        # 6. Author
        author_meta = soup.find("meta", {"name": "author"})
        author = author_meta["content"].strip() if author_meta else ""

        # 7. Thumbnail
        img_meta = soup.find("meta", {"property": "og:image"})
        thumbnail = img_meta["content"] if img_meta else ""

        # 8. Word count
        word_count = len(content.split()) if content else 0

        return {
            "title": title,
            "url": url,
            "publish_date": publish_date,
            "category": category,
            "summary": summary,
            "content": content,
            "word_count": word_count,
            "tags": tags,
            "author": author,
            "thumbnail": thumbnail,
            "crawled_at": datetime.now().isoformat()
        }

    except Exception as e:
        print(f"Lỗi khi crawl {url}: {e}")
        return None

def get_article_urls(category_name, category_url, pages=15):
    """Lấy danh sách URL bài viết từ một danh mục"""
    urls = set()
    successful_pages = 0
    
    print(f"\n Bắt đầu crawl danh mục: {category_name.upper()}")
    
    for page in range(1, pages + 1):
        page_url = f"{category_url}{page}"
        try:
            r = requests.get(page_url, headers=headers, timeout=10)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            
            # Các selector cho link bài viết
            link_selectors = [
                "h3.title-news a[href]",
                "h2.title-news a[href]",
                "h4.title-news a[href]",
                "article h3 a[href]"
            ]
            
            page_urls = set()
            for selector in link_selectors:
                for link in soup.select(selector):
                    href = link.get("href", "")
                    if href:
                        if href.startswith("/"):
                            href = urljoin("https://vnexpress.net", href)
                        if "vnexpress.net" in href and href not in urls:
                            page_urls.add(href)
            
            urls.update(page_urls)
            successful_pages += 1
            print(f"   Trang {page}: +{len(page_urls)} bài ({len(urls)} tổng)")
            
            # Delay để tránh bị chặn
            time.sleep(1)
            
        except Exception as e:
            print(f"   Lỗi trang {page}: {e}")
    
    print(f"Hoàn thành {category_name}: {len(urls)} bài từ {successful_pages}/{pages} trang")
    return list(urls)

def crawl_category(category_name, save_individual=True):
    """Crawl một danh mục và có thể lưu file riêng"""
    if category_name not in categories:
        print(f" Danh mục '{category_name}' không tồn tại!")
        return []
    
    category_url = categories[category_name]
    
    # Lấy URLs
    article_urls = get_article_urls(category_name, category_url, NUM_PAGES)
    
    if not article_urls:
        print(f" Không lấy được URL nào từ {category_name}")
        return []
    
    # Crawl từng bài viết
    category_results = []
    print(f"\n Bắt đầu crawl {len(article_urls)} bài viết...")
    
    for i, url in enumerate(article_urls, 1):
        print(f"   [{i}/{len(article_urls)}] Crawling: {url}")
        
        article = extract_article(url, category_name)
        if article:
            category_results.append(article)
            print(f"    Thành công: {article['title'][:50]}...")
        else:
            print(f"    Thất bại")
        
        # Delay để tránh bị chặn  
        time.sleep(0.8)
    
    # Lưu file riêng nếu cần
    if save_individual and category_results:
        filename = f"vnexpress_{category_name}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(category_results, f, ensure_ascii=False, indent=2)
        print(f" Đã lưu {len(category_results)} bài {category_name} vào {filename}")
    
    # Thêm vào kết quả tổng
    all_results.extend(category_results)
    
    print(f"Hoàn thành crawl {category_name}: {len(category_results)} bài thành công")
    return category_results

def save_all_results(filename="vnexpress_all_categories.json"):
    """Lưu tất cả kết quả vào một file"""
    if all_results:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)
        print(f"\n ĐÃ LƯU TẤT CẢ: {len(all_results)} bài vào {filename}")
    else:
        print(" Chưa có dữ liệu nào để lưu")

def get_statistics():
    """Hiển thị thống kê crawl"""
    if not all_results:
        print(" Chưa có dữ liệu để thống kê")
        return
    
    # Thống kê theo danh mục
    category_stats = {}
    total_words = 0
    
    for article in all_results:
        cat = article['category']
        if cat not in category_stats:
            category_stats[cat] = {'count': 0, 'words': 0}
        category_stats[cat]['count'] += 1
        category_stats[cat]['words'] += article.get('word_count', 0)
        total_words += article.get('word_count', 0)
    
    print(f"\n THỐNG KÊ CRAWL:")
    print(f"    Tổng số bài viết: {len(all_results)}")
    print(f"    Tổng số từ: {total_words:,}")
    print(f"   Trung bình từ/bài: {total_words/len(all_results):.0f}")
    print(f"\n Theo danh mục:")
    
    for cat, stats in sorted(category_stats.items()):
        avg_words = stats['words'] / stats['count'] if stats['count'] > 0 else 0
        print(f"   {cat:12}: {stats['count']:3d} bài ({stats['words']:6,} từ, TB: {avg_words:.0f})")

print(f" Danh sách categories: {list(categories.keys())}")



 Danh sách categories: ['thoi-su', 'the-gioi', 'kinh-doanh', 'giai-tri', 'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe', 'doi-song', 'du-lich', 'khoa-hoc', 'so-hoa', 'oto-xe-may', 'y-kien', 'tam-su', 'cuoi']


In [ ]:

def extract_article_improved(url, category, max_retries=3):
    """Trích xuất nội dung với retry logic xử lý timeout"""
    for attempt in range(max_retries):
        try:
            print(f"     Thử lần {attempt + 1}/{max_retries}...")
            
            # Tăng timeout và thêm retry logic
            r = requests.get(url, headers=headers, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")

            # 1. Title
            title_elem = soup.find("h1", class_="title-detail") or soup.title
            title = title_elem.get_text().strip() if title_elem else ""

            # 2. Summary/Description
            meta_desc = soup.find("meta", {"name": "description"})
            summary = meta_desc["content"].strip() if meta_desc else ""
            
            if not summary:
                sapo_elem = soup.find("p", class_="description")
                summary = sapo_elem.get_text().strip() if sapo_elem else ""

            # 3. Content
            content = ""
            content_selectors = [
                "article.fck_detail",
                "div.fck_detail", 
                "div.sidebar-1",
                "article.content_detail"
            ]
            
            for selector in content_selectors:
                content_div = soup.select_one(selector)
                if content_div:
                    paragraphs = content_div.find_all(["p", "h3", "h4", "li"])
                    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
                    break

            # 4. Publish date
            pub_meta = soup.find("meta", {"itemprop": "datePublished"})
            if not pub_meta:
                pub_meta = soup.find("meta", {"property": "article:published_time"})
            publish_date = pub_meta["content"] if pub_meta else ""

            # 5. Tags - CHỈ LẤY KEYWORDS (không lấy breadcrumb)
            tags = []
            
            # Meta article:tag
            tag_metas = soup.find_all("meta", {"property": "article:tag"})
            for meta in tag_metas:
                if "content" in meta.attrs:
                    tags.append(meta["content"].strip())
            
            # Meta keywords (chính xác nhất)
            keywords_meta = soup.find("meta", {"name": "keywords"})
            if keywords_meta and "content" in keywords_meta.attrs:
                keywords = [k.strip() for k in keywords_meta["content"].split(",")]
                tags.extend(keywords)
            
            # Meta news_keywords  
            news_keywords_meta = soup.find("meta", {"name": "news_keywords"})
            if news_keywords_meta and "content" in news_keywords_meta.attrs:
                news_keywords = [k.strip() for k in news_keywords_meta["content"].split(",")]
                tags.extend(news_keywords)
            
            # Tags từ div.tags
            tag_divs = soup.select("div.tags a, div.tag-list a, .article-tags a")
            for tag_div in tag_divs:
                if tag_div.get_text().strip():
                    tags.append(tag_div.get_text().strip())
            
            # Làm sạch tags (loại bỏ breadcrumb)
            tags = list(set([tag for tag in tags if tag and len(tag.strip()) > 0]))
            tags = [tag for tag in tags if not tag.lower() in ["vnexpress", "vietnam", "việt nam", "trang chủ", "home"]]

            # 6. Author
            author_meta = soup.find("meta", {"name": "author"})
            author = author_meta["content"].strip() if author_meta else ""

            # 7. Thumbnail
            img_meta = soup.find("meta", {"property": "og:image"})
            thumbnail = img_meta["content"] if img_meta else ""

            # 8. Word count
            word_count = len(content.split()) if content else 0

            return {
                "title": title,
                "url": url,
                "publish_date": publish_date,
                "category": category,
                "summary": summary,
                "content": content,
                "word_count": word_count,
                "tags": tags,
                "author": author,
                "thumbnail": thumbnail,
                "crawled_at": datetime.now().isoformat()
            }

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f"      Lần {attempt + 1}: Timeout/Connection error")
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 3  # 3s, 6s, 9s
                print(f"      Chờ {wait_time}s trước khi thử lại...")
                time.sleep(wait_time)
            continue
        except requests.exceptions.HTTPError as e:
            print(f"      Lần {attempt + 1}: HTTP Error {e.response.status_code}")
            if e.response.status_code == 429:  # Rate limited
                wait_time = (attempt + 1) * 5  # 5s, 10s, 15s
                print(f"    Rate limited! Chờ {wait_time}s...")
                time.sleep(wait_time)
                continue
            break  # Không retry cho HTTP errors khác
        except Exception as e:
            print(f"       Lần {attempt + 1}: Lỗi khác - {str(e)[:100]}")
            if attempt < max_retries - 1:
                time.sleep(2)
            continue
    
    print(f"     Đã thử {max_retries} lần nhưng vẫn thất bại!")
    return None

def crawl_category_improved(category_name, save_individual=True):
    """Crawl với error handling tốt hơn"""
    if category_name not in categories:
        print(f" Danh mục '{category_name}' không tồn tại!")
        return []
    
    category_url = categories[category_name]
    
    # Lấy URLs
    article_urls = get_article_urls(category_name, category_url, NUM_PAGES)
    
    if not article_urls:
        print(f" Không lấy được URL nào từ {category_name}")
        return []
    
    # Crawl từng bài viết
    category_results = []
    successful = 0
    failed = 0
    
    print(f"\n Bắt đầu crawl {len(article_urls)} bài viết...")
    
    for i, url in enumerate(article_urls, 1):
        print(f"   [{i}/{len(article_urls)}] Crawling: {url}")
        
        article = extract_article_improved(url, category_name)
        if article:
            category_results.append(article)
            successful += 1
            print(f"       Thành công: {article['title'][:40]}...")
        else:
            failed += 1
            print(f"       Thất bại")
        
        # Delay để tránh bị chặn  
        time.sleep(1.2)  # Tăng delay
        
        # Báo cáo tiến độ mỗi 10 bài
        if i % 10 == 0:
            print(f"    Tiến độ: {successful} thành công, {failed} thất bại")
    
    # Lưu file riêng nếu cần
    if save_individual and category_results:
        filename = f"vnexpress_{category_name}_improved.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(category_results, f, ensure_ascii=False, indent=2)
        print(f" Đã lưu {len(category_results)} bài {category_name} vào {filename}")
    
    # Thêm vào kết quả tổng
    all_results.extend(category_results)
    
    print(f" Hoàn thành crawl {category_name}: {successful} thành công, {failed} thất bại")
    return category_results



In [6]:
# CRAWL THỜI SỰ (Current Affairs)

thoi_su_data = crawl_category("thoi-su")
get_statistics()



 Bắt đầu crawl danh mục: THOI-SU
   Trang 1: +50 bài (50 tổng)
   Trang 2: +29 bài (79 tổng)
   Trang 3: +29 bài (108 tổng)
   Trang 4: +29 bài (137 tổng)
   Trang 5: +29 bài (166 tổng)
   Trang 6: +29 bài (195 tổng)
   Trang 7: +29 bài (224 tổng)
   Trang 8: +29 bài (253 tổng)
   Trang 9: +29 bài (282 tổng)
   Trang 10: +29 bài (311 tổng)
   Trang 11: +29 bài (340 tổng)
   Trang 12: +29 bài (369 tổng)
   Trang 13: +29 bài (398 tổng)
   Trang 14: +29 bài (427 tổng)
   Trang 15: +29 bài (456 tổng)
Hoàn thành thoi-su: 456 bài từ 15/15 trang

 Bắt đầu crawl 456 bài viết...
   [1/456] Crawling: https://vnexpress.net/ap-thap-nhiet-doi-hinh-thanh-tren-bien-dong-4909932.html
    Thành công: Áp thấp nhiệt đới hình thành trên Biển Đông...
   [2/456] Crawling: https://vnexpress.net/ha-noi-sat-hach-giay-phep-lai-xe-tu-ngay-mai-4905269.html
    Thành công: Hà Nội sát hạch giấy phép lái xe từ ngày mai...
   [3/456] Crawling: https://vnexpress.net/tau-khach-noi-them-3-toa-khi-san-bay-vinh-dong-cua-

In [7]:
# CRAWL THẾ GIỚI (World News)

the_gioi_data = crawl_category("the-gioi")
get_statistics()



 Bắt đầu crawl danh mục: THE-GIOI
   Trang 1: +50 bài (50 tổng)
   Trang 2: +29 bài (79 tổng)
   Trang 3: +29 bài (108 tổng)
   Trang 4: +29 bài (137 tổng)
   Trang 5: +29 bài (166 tổng)
   Trang 6: +29 bài (195 tổng)
   Trang 7: +29 bài (224 tổng)
   Trang 8: +29 bài (253 tổng)
   Trang 9: +29 bài (282 tổng)
   Trang 10: +29 bài (311 tổng)
   Trang 11: +29 bài (340 tổng)
   Trang 12: +29 bài (369 tổng)
   Trang 13: +29 bài (398 tổng)
   Trang 14: +29 bài (427 tổng)
   Trang 15: +29 bài (456 tổng)
Hoàn thành the-gioi: 456 bài từ 15/15 trang

 Bắt đầu crawl 456 bài viết...
   [1/456] Crawling: https://vnexpress.net/hai-dao-cua-nhat-tach-xa-nhau-gan-10-cm-trong-ba-ngay-4913030.html
    Thành công: Hai đảo của Nhật 'tách xa nhau gần 10 cm trong ba ...
   [2/456] Crawling: https://vnexpress.net/vong-vay-cua-israel-bop-nghet-nguon-song-cho-tre-so-sinh-gaza-4907053.html
    Thành công: Vòng vây của Israel bóp nghẹt nguồn sống cho trẻ s...
   [3/456] Crawling: https://vnexpress.net/nhung-ngu

In [4]:
# CRAWL PHÁP LUẬT (Law)

phap_luat_data = crawl_category("phap-luat")
get_statistics()



 Bắt đầu crawl danh mục: PHAP-LUAT
   Trang 1: +50 bài (50 tổng)
   Trang 2: +29 bài (79 tổng)
   Trang 3: +29 bài (108 tổng)
   Trang 4: +29 bài (137 tổng)
   Trang 5: +29 bài (166 tổng)
   Trang 6: +29 bài (195 tổng)
   Trang 7: +29 bài (224 tổng)
   Trang 8: +29 bài (253 tổng)
   Trang 9: +29 bài (282 tổng)
   Trang 10: +29 bài (311 tổng)
   Trang 11: +29 bài (340 tổng)
   Trang 12: +29 bài (369 tổng)
   Trang 13: +29 bài (398 tổng)
   Trang 14: +29 bài (427 tổng)
   Trang 15: +29 bài (456 tổng)
Hoàn thành phap-luat: 456 bài từ 15/15 trang

 Bắt đầu crawl 456 bài viết...
   [1/456] Crawling: https://vnexpress.net/ba-truong-my-lan-va-doi-tac-len-phuong-an-khac-phuc-3-ty-usd-4904085.html
    Thành công: Bà Trương Mỹ Lan và đối tác lên phương án khắc phụ...
   [2/456] Crawling: https://vnexpress.net/lua-nang-diem-gia-200-trieu-dong-4898583.html
    Thành công: Lừa nâng điểm giá 200 triệu đồng...
   [3/456] Crawling: https://vnexpress.net/thi-the-bi-troi-hai-tay-mac-vao-luoi-danh-ca-49

In [5]:
#  CRAWL KINH DOANH (Business)

kinh_doanh_data = crawl_category("kinh-doanh")
get_statistics()



 Bắt đầu crawl danh mục: KINH-DOANH
   Trang 1: +53 bài (53 tổng)
   Trang 2: +30 bài (83 tổng)
   Trang 3: +30 bài (113 tổng)
   Trang 4: +30 bài (143 tổng)
   Trang 5: +30 bài (173 tổng)
   Trang 6: +30 bài (203 tổng)
   Trang 7: +30 bài (233 tổng)
   Trang 8: +30 bài (263 tổng)
   Trang 9: +30 bài (293 tổng)
   Trang 10: +30 bài (323 tổng)
   Trang 11: +30 bài (353 tổng)
   Trang 12: +30 bài (383 tổng)
   Trang 13: +30 bài (413 tổng)
   Trang 14: +30 bài (443 tổng)
   Trang 15: +30 bài (473 tổng)
Hoàn thành kinh-doanh: 473 bài từ 15/15 trang

 Bắt đầu crawl 473 bài viết...
   [1/473] Crawling: https://vnexpress.net/ty-phu-pham-nhat-vuong-tiep-tuc-rot-von-cho-vinspeed-4910662.html
    Thành công: Tỷ phú Phạm Nhật Vượng tiếp tục rót vốn cho VinSpe...
   [2/473] Crawling: https://vnexpress.net/opec-lien-tuc-nang-san-luong-dau-4910677.html
    Thành công: OPEC+ liên tục nâng sản lượng dầu...
   [3/473] Crawling: https://vnexpress.net/san-bay-vinh-dong-cua-hanh-khach-se-duoc-hoan-ve-mie

In [4]:
# CRAWL GIẢI TRÍ (Entertainment)

giai_tri_data = crawl_category("giai-tri")
get_statistics()



 Bắt đầu crawl danh mục: GIAI-TRI
   Trang 1: +51 bài (51 tổng)
   Trang 2: +30 bài (81 tổng)
   Trang 3: +30 bài (111 tổng)
   Trang 4: +30 bài (141 tổng)
   Trang 5: +30 bài (171 tổng)
   Trang 6: +30 bài (201 tổng)
   Trang 7: +30 bài (231 tổng)
   Trang 8: +30 bài (261 tổng)
   Trang 9: +30 bài (291 tổng)
   Trang 10: +30 bài (321 tổng)
   Trang 11: +30 bài (351 tổng)
   Trang 12: +30 bài (381 tổng)
   Trang 13: +30 bài (411 tổng)
   Trang 14: +30 bài (441 tổng)
   Trang 15: +30 bài (471 tổng)
Hoàn thành giai-tri: 471 bài từ 15/15 trang

 Bắt đầu crawl 471 bài viết...
   [1/471] Crawling: https://vnexpress.net/le-thanh-hon-cua-ty-phu-jeff-bezos-4907335.html
    Thành công: Lễ thành hôn của tỷ phú Jeff Bezos...
   [2/471] Crawling: https://vnexpress.net/trang-phuc-ton-voc-dang-madam-pang-4903570.html
    Thành công: Trang phục tôn vóc dáng Madam Pang...
   [3/471] Crawling: https://vnexpress.net/phuong-my-chi-hat-ly-bac-bo-o-sing-asia-4898344.html
    Thành công: Phương Mỹ Chi hát 

In [6]:
# CRAWL THỂ THAO (Sports)

the_thao_data = crawl_category("the-thao")
get_statistics()



 Bắt đầu crawl danh mục: THE-THAO
   Trang 1: +67 bài (67 tổng)
   Trang 2: +30 bài (97 tổng)
   Trang 3: +29 bài (126 tổng)
   Trang 4: +30 bài (156 tổng)
   Trang 5: +30 bài (186 tổng)
   Trang 6: +30 bài (216 tổng)
   Trang 7: +30 bài (246 tổng)
   Trang 8: +30 bài (276 tổng)
   Trang 9: +30 bài (306 tổng)
   Trang 10: +30 bài (336 tổng)
   Trang 11: +30 bài (366 tổng)
   Trang 12: +30 bài (396 tổng)
   Trang 13: +30 bài (426 tổng)
   Trang 14: +30 bài (456 tổng)
   Trang 15: +30 bài (486 tổng)
Hoàn thành the-thao: 486 bài từ 15/15 trang

 Bắt đầu crawl 486 bài viết...
   [1/486] Crawling: https://vnexpress.net/cau-thu-bong-ro-goc-viet-nhan-hop-dong-24-trieu-usd-4908238.html
    Thành công: Cầu thủ bóng rổ gốc Việt nhận hợp đồng 24 triệu US...
   [2/486] Crawling: https://vnexpress.net/boston-marathon-siet-chuan-voi-runner-tu-cac-giai-do-doc-4905037.html
    Thành công: Boston Marathon siết chuẩn với runner từ các giải ...
   [3/486] Crawling: https://vnexpress.net/quang-liem-thang

In [5]:
#  CRAWL GIÁO DỤC (Education)

giao_duc_data = crawl_category("giao-duc")
get_statistics()



 Bắt đầu crawl danh mục: GIAO-DUC
   Trang 1: +54 bài (54 tổng)
   Trang 2: +29 bài (83 tổng)
   Trang 3: +29 bài (112 tổng)
   Trang 4: +29 bài (141 tổng)
   Trang 5: +29 bài (170 tổng)
   Trang 6: +29 bài (199 tổng)
   Trang 7: +29 bài (228 tổng)
   Trang 8: +29 bài (257 tổng)
   Trang 9: +29 bài (286 tổng)
   Trang 10: +29 bài (315 tổng)
   Trang 11: +29 bài (344 tổng)
   Trang 12: +29 bài (373 tổng)
   Trang 13: +29 bài (402 tổng)
   Trang 14: +29 bài (431 tổng)
   Trang 15: +29 bài (460 tổng)
Hoàn thành giao-duc: 460 bài từ 15/15 trang

 Bắt đầu crawl 460 bài viết...
   [1/460] Crawling: https://vnexpress.net/thu-khoa-tu-can-nha-20-m2-4911947.html
    Thành công: Thủ khoa từ căn nhà 20 m2...
   [2/460] Crawling: https://vnexpress.net/du-doan-pho-diem-mon-van-thi-tot-nghiep-thpt-2025-4906403.html
    Thành công: Đề Văn thi tốt nghiệp 'ấn tượng', dự đoán nhiều đi...
   [3/460] Crawling: https://vnexpress.net/hoc-phi-dai-hoc-su-pham-ky-thuat-tp-hcm-2025-chi-tiet-tung-nganh-4913414.h

In [7]:
# CRAWL SỨC KHỎE (Health)

suc_khoe_data = crawl_category("suc-khoe")
get_statistics()



 Bắt đầu crawl danh mục: SUC-KHOE
   Trang 1: +53 bài (53 tổng)
   Trang 2: +30 bài (83 tổng)
   Trang 3: +30 bài (113 tổng)
   Trang 4: +30 bài (143 tổng)
   Trang 5: +30 bài (173 tổng)
   Trang 6: +30 bài (203 tổng)
   Trang 7: +30 bài (233 tổng)
   Trang 8: +30 bài (263 tổng)
   Trang 9: +30 bài (293 tổng)
   Trang 10: +30 bài (323 tổng)
   Trang 11: +30 bài (353 tổng)
   Trang 12: +30 bài (383 tổng)
   Trang 13: +30 bài (413 tổng)
   Trang 14: +30 bài (443 tổng)
   Trang 15: +30 bài (473 tổng)
Hoàn thành suc-khoe: 473 bài từ 15/15 trang

 Bắt đầu crawl 473 bài viết...
   [1/473] Crawling: https://vnexpress.net/bong-den-no-vang-manh-thuy-tinh-vao-mat-gay-thung-nhan-cau-4911679.html
    Thành công: Bóng đèn nổ văng mảnh thủy tinh vào mắt gây thủng ...
   [2/473] Crawling: https://vnexpress.net/vi-sao-da-chay-xe-sau-giam-can-4907414.html
    Thành công: Vì sao da chảy xệ sau giảm cân?...
   [3/473] Crawling: https://vnexpress.net/be-gai-viem-tuy-man-tinh-4908061.html
    Thành công: 

In [ ]:
# CRAWL ĐỜI SỐNG (Lifestyle)

doi_song_data = crawl_category("doi-song")
get_statistics()



 Bắt đầu crawl danh mục: DOI-SONG
   Trang 1: +47 bài (47 tổng)
   Trang 2: +30 bài (77 tổng)
   Trang 3: +30 bài (107 tổng)
   Trang 4: +30 bài (137 tổng)
   Trang 5: +30 bài (167 tổng)
   Trang 6: +30 bài (197 tổng)
   Trang 7: +30 bài (227 tổng)
   Trang 8: +30 bài (257 tổng)
   Trang 9: +30 bài (287 tổng)
   Trang 10: +30 bài (317 tổng)
   Trang 11: +30 bài (347 tổng)
   Trang 12: +30 bài (377 tổng)
   Trang 13: +30 bài (407 tổng)
   Trang 14: +30 bài (437 tổng)
   Trang 15: +30 bài (467 tổng)
Hoàn thành doi-song: 467 bài từ 15/15 trang

 Bắt đầu crawl 467 bài viết...
   [1/467] Crawling: https://vnexpress.net/xay-phong-lop-moi-cho-hoc-sinh-mien-nui-quang-nam-4903817.html
    Thành công: Xây phòng lớp mới cho học sinh miền núi Quảng Nam...
   [2/467] Crawling: https://vnexpress.net/ngoi-lang-tang-nha-100-m2-cho-thanh-nien-4892261.html
    Thành công: Ngôi làng tặng nhà 100 m2 cho thanh niên...
   [3/467] Crawling: https://vnexpress.net/quan-he-tinh-duc-nhieu-co-giup-vo-chong-hanh-

In [ ]:
# CRAWL DU LỊCH (Travel)

du_lich_data = crawl_category("du-lich")
get_statistics()


In [ ]:
# CRAWL KHOA HỌC (Science)

khoa_hoc_data = crawl_category("khoa-hoc")
get_statistics()


In [ ]:
# CRAWL SỐ HÓA (Digital Technology)

so_hoa_data = crawl_category("so-hoa")
get_statistics()


In [ ]:
# CRAWL OTO XE MÁY (Automotive)

oto_xe_may_data = crawl_category("oto-xe-may")
get_statistics()


In [ ]:
# CRAWL Ý KIẾN (Opinion)

y_kien_data = crawl_category("y-kien")
get_statistics()


In [ ]:
# CRAWL TÂM SỰ (Confession)

tam_su_data = crawl_category("tam-su")
get_statistics()


In [ ]:
#  CRAWL HÀI HƯỚC/CƯỜI (Humor)

cuoi_data = crawl_category("cuoi")
get_statistics()


In [ ]:
#  LƯU TẤT CẢ DỮ LIỆU & THỐNG KÊ TỔNG KẾT

# Lưu tất cả dữ liệu vào 1 file tổng
save_all_results("vnexpress_all_16_categories.json")

# Thống kê tổng kết cuối cùng
print("="*60)
print(" TỔNG KẾT CRAWL TẤT CẢ 16 DANH MỤC VNEXPRESS")
print("="*60)
get_statistics()

# Tạo file CSV tổng hợp (nếu cần)
if all_results:
    import pandas as pd
    
    # Tạo DataFrame đơn giản
    df_data = []
    for article in all_results:
        df_data.append({
            'category': article['category'],
            'title': article['title'],
            'url': article['url'],
            'publish_date': article['publish_date'],
            'word_count': article['word_count'],
            'author': article['author'],
            'tags_count': len(article['tags']),
            'has_thumbnail': bool(article['thumbnail'])
        })
    
    df = pd.DataFrame(df_data)
    df.to_csv("vnexpress_all_categories.csv", index=False, encoding='utf-8-sig')
    print(f"\n Đã xuất CSV: vnexpress_all_categories.csv")
print(f"\n HOÀN THÀNH CRAWL TẤT CẢ 16 DANH MỤC!")


In [1]:
# 🛠️ FUNCTION CẢI TIẾN XỬ LÝ LỖI TIMEOUT

def extract_article_improved(url, category, max_retries=3):
    """Trích xuất nội dung với retry logic xử lý timeout"""
    for attempt in range(max_retries):
        try:
            print(f"      🔄 Thử lần {attempt + 1}/{max_retries}...")
            
            # Tăng timeout và thêm retry logic
            r = requests.get(url, headers=headers, timeout=30)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")

            # 1. Title
            title_elem = soup.find("h1", class_="title-detail") or soup.title
            title = title_elem.get_text().strip() if title_elem else ""

            # 2. Summary/Description
            meta_desc = soup.find("meta", {"name": "description"})
            summary = meta_desc["content"].strip() if meta_desc else ""
            
            if not summary:
                sapo_elem = soup.find("p", class_="description")
                summary = sapo_elem.get_text().strip() if sapo_elem else ""

            # 3. Content
            content = ""
            content_selectors = [
                "article.fck_detail",
                "div.fck_detail", 
                "div.sidebar-1",
                "article.content_detail"
            ]
            
            for selector in content_selectors:
                content_div = soup.select_one(selector)
                if content_div:
                    paragraphs = content_div.find_all(["p", "h3", "h4", "li"])
                    content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
                    break

            # 4. Publish date
            pub_meta = soup.find("meta", {"itemprop": "datePublished"})
            if not pub_meta:
                pub_meta = soup.find("meta", {"property": "article:published_time"})
            publish_date = pub_meta["content"] if pub_meta else ""

            # 5. Tags - CHỈ LẤY KEYWORDS (không lấy breadcrumb)
            tags = []
            
            # Meta article:tag
            tag_metas = soup.find_all("meta", {"property": "article:tag"})
            for meta in tag_metas:
                if "content" in meta.attrs:
                    tags.append(meta["content"].strip())
            
            # Meta keywords (chính xác nhất)
            keywords_meta = soup.find("meta", {"name": "keywords"})
            if keywords_meta and "content" in keywords_meta.attrs:
                keywords = [k.strip() for k in keywords_meta["content"].split(",")]
                tags.extend(keywords)
            
            # Meta news_keywords  
            news_keywords_meta = soup.find("meta", {"name": "news_keywords"})
            if news_keywords_meta and "content" in news_keywords_meta.attrs:
                news_keywords = [k.strip() for k in news_keywords_meta["content"].split(",")]
                tags.extend(news_keywords)
            
            # Tags từ div.tags
            tag_divs = soup.select("div.tags a, div.tag-list a, .article-tags a")
            for tag_div in tag_divs:
                if tag_div.get_text().strip():
                    tags.append(tag_div.get_text().strip())
            
            # Làm sạch tags (loại bỏ breadcrumb)
            tags = list(set([tag for tag in tags if tag and len(tag.strip()) > 0]))
            tags = [tag for tag in tags if not tag.lower() in ["vnexpress", "vietnam", "việt nam", "trang chủ", "home"]]

            # 6. Author
            author_meta = soup.find("meta", {"name": "author"})
            author = author_meta["content"].strip() if author_meta else ""

            # 7. Thumbnail
            img_meta = soup.find("meta", {"property": "og:image"})
            thumbnail = img_meta["content"] if img_meta else ""

            # 8. Word count
            word_count = len(content.split()) if content else 0

            return {
                "title": title,
                "url": url,
                "publish_date": publish_date,
                "category": category,
                "summary": summary,
                "content": content,
                "word_count": word_count,
                "tags": tags,
                "author": author,
                "thumbnail": thumbnail,
                "crawled_at": datetime.now().isoformat()
            }

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f"      ⚠️ Lần {attempt + 1}: Timeout/Connection error")
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 3  # 3s, 6s, 9s
                print(f"      ⏳ Chờ {wait_time}s trước khi thử lại...")
                time.sleep(wait_time)
            continue
        except requests.exceptions.HTTPError as e:
            print(f"      ❌ Lần {attempt + 1}: HTTP Error {e.response.status_code}")
            if e.response.status_code == 429:  # Rate limited
                wait_time = (attempt + 1) * 5  # 5s, 10s, 15s
                print(f"      🚫 Rate limited! Chờ {wait_time}s...")
                time.sleep(wait_time)
                continue
            break  # Không retry cho HTTP errors khác
        except Exception as e:
            print(f"      💥 Lần {attempt + 1}: Lỗi khác - {str(e)[:100]}")
            if attempt < max_retries - 1:
                time.sleep(2)
            continue
    
    print(f"      💀 Đã thử {max_retries} lần nhưng vẫn thất bại!")
    return None

def crawl_category_improved(category_name, save_individual=True):
    """Crawl với error handling tốt hơn"""
    if category_name not in categories:
        print(f"❌ Danh mục '{category_name}' không tồn tại!")
        return []
    
    category_url = categories[category_name]
    
    # Lấy URLs
    article_urls = get_article_urls(category_name, category_url, NUM_PAGES)
    
    if not article_urls:
        print(f"❌ Không lấy được URL nào từ {category_name}")
        return []
    
    # Crawl từng bài viết
    category_results = []
    successful = 0
    failed = 0
    
    print(f"\n📝 Bắt đầu crawl {len(article_urls)} bài viết...")
    
    for i, url in enumerate(article_urls, 1):
        print(f"   [{i}/{len(article_urls)}] Crawling: {url}")
        
        article = extract_article_improved(url, category_name)
        if article:
            category_results.append(article)
            successful += 1
            print(f"      ✅ Thành công: {article['title'][:40]}...")
        else:
            failed += 1
            print(f"      ❌ Thất bại")
        
        # Delay để tránh bị chặn  
        time.sleep(1.2)  # Tăng delay
        
        # Báo cáo tiến độ mỗi 10 bài
        if i % 10 == 0:
            print(f"   📊 Tiến độ: {successful} thành công, {failed} thất bại")
    
    # Lưu file riêng nếu cần
    if save_individual and category_results:
        filename = f"vnexpress_{category_name}_improved.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(category_results, f, ensure_ascii=False, indent=2)
        print(f"💾 Đã lưu {len(category_results)} bài {category_name} vào {filename}")
    
    # Thêm vào kết quả tổng
    all_results.extend(category_results)
    
    print(f"🎉 Hoàn thành crawl {category_name}: {successful} thành công, {failed} thất bại")
    return category_results

print("🔧 Các function cải tiến đã sẵn sàng!")
print("💡 Sử dụng: crawl_category_improved('ten-danh-muc') thay vì crawl_category()")


🔧 Các function cải tiến đã sẵn sàng!
💡 Sử dụng: crawl_category_improved('ten-danh-muc') thay vì crawl_category()
